In [1]:
import numpy as np
import pandas as pd
from scipy.stats import entropy
import matplotlib.pyplot as plt
import random

import cornac
from cornac.eval_methods import RatioSplit
from cornac.data import TextModality
from cornac.data.text import BaseTokenizer
from cornac.data import Reader
from cornac.metrics import MAE,RMSE,MSE,FMeasure,Precision,Recall,NDCG,NCRR,MRR,AUC,MAP

FM model is only supported on Linux.
Windows executable can be found at http://www.libfm.org.


In [2]:
docs_movielens, item_ids_movielens = cornac.datasets.movielens.load_plot()
feedback_movielens_100K = cornac.datasets.movielens.load_feedback(variant="100K",reader=Reader(item_set=item_ids_movielens))

movielens_item_text_modality = TextModality(
    corpus=docs_movielens,
    ids=item_ids_movielens,
    tokenizer=BaseTokenizer(sep="\t", stop_words="english"),
    max_vocab=8000,
    max_doc_freq=0.5,
)
# Define an evaluation method to split feedback into train and test sets
movieslens100K_ratio_split = RatioSplit(
    data=feedback_movielens_100K,
    test_size=0.2,
    exclude_unknowns=True,
    item_text=movielens_item_text_modality,
    verbose=True,
    seed=123,
    rating_threshold=1,
)

rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 943
Number of items = 1518
Number of ratings = 75846
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 941
Number of items = 1279
Number of ratings = 18925
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 943
Total items = 1518


### k=10

In [3]:
# Instantiate HFT model
ctr_k10 = cornac.models.CTR(
    k=10,
    lambda_u=0.01,
    lambda_v=0.01,
    eta=0.01,
    a=1,
    b=0.01,
    max_iter=10,
    trainable=True,
    verbose=True,
    init_params=None,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[ctr_k10], metrics=metrics, user_based=False
).run()


[CTR] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[CTR] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
CTR | 0.8401 | 1.2007 | 1.0957 | 0.9006 | 0.1163 | 0.1025 | 0.2653 |  0.1364 |  0.1979 |       0.0842 |    0.3199 |    4.4037 |   2.8740



In [4]:
from cornac.utils import cache

# Download some information of MovieLens 100K dataset
user_df = pd.read_csv(
  cache("http://files.grouplens.org/datasets/movielens/ml-100k/u.user"),
  sep="|", names=["UserID", "Age", "Gender", "Occupation", "Zip Code"]
).set_index("UserID")

item_df = pd.read_csv(
  cache("http://files.grouplens.org/datasets/movielens/ml-100k/u.item"),
  sep="|", encoding="ISO-8859-1",
  names=["ItemID", "Title", "Release Date", "Video Release Date", "IMDb URL",
         "unknown", "Action", "Adventure", "Animation", "Children's", "Comedy",
         "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
         "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]
).set_index("ItemID").drop(columns=["Video Release Date", "IMDb URL", "unknown"])

In [5]:
import diversity_movielens
def compute_avg_diversity(model):
    for i in range(len(set(movieslens100K_ratio_split.test_set.user_ids))):  # get the average result for all users
        result = []
        r = diversity_movielens.MovielensRetrieval(model=model, item_df=item_df, UIDX=i, TOPK=50)
        d = diversity_movielens.Diversity()
        recommendation = d.compute_dict_distr(r.get_recy())
        history = d.compute_dict_distr(r.get_history())
        result.append(d.compute_kl_divergence(recommendation, history, alpha=0.001))
    return np.mean(result)

In [6]:
data1 = {"model":"CTR",
        "data":"movielens-100K",
        "k":10,
        "lambda_u":0.01,
        "lambda_v":0.01,
        "eta":0.01,
        "a":1,
        "b":0.01,
        "max_iter":10,
        "MAE":0.8401,"MSE":1.2007,"RMSE":1.0957,"AUC":0.9006,"F1":0.1163,"MAP":0.1025,"MRR":0.2653,"NCRR":0.1364,"NDCG":0.1979,"Precision":0.0842,"Recall":0.3199,
        "category diversity":compute_avg_diversity(ctr_k10)}
data1["category diversity"]

0.10033744023997579

### k=50

In [7]:
# Instantiate HFT model
ctr_k50 = cornac.models.CTR(
    k=50,
    lambda_u=0.01,
    lambda_v=0.01,
    eta=0.01,
    a=1,
    b=0.01,
    max_iter=10,
    trainable=True,
    verbose=True,
    init_params=None,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[ctr_k50], metrics=metrics, user_based=False
).run()


[CTR] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[CTR] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
CTR | 1.2712 | 2.5926 | 1.6102 | 0.8651 | 0.0976 | 0.0810 | 0.2618 |  0.1217 |  0.1627 |       0.0722 |    0.2631 |   11.1496 |   1.5158



In [8]:
data2 = {"model":"CTR",
        "data":"movielens-100K",
        "k":50,
        "lambda_u":0.01,
        "lambda_v":0.01,
        "eta":0.01,
        "a":1,
        "b":0.01,
        "max_iter":10,
        "MAE":1.2712,"MSE":2.5926,"RMSE":1.6102,"AUC":0.8651,"F1":0.0976,"MAP":0.0810,"MRR":0.2618,"NCRR":0.1217,"NDCG":0.1627,"Precision":0.0722,"Recall":0.2631,
        "category diversity":compute_avg_diversity(ctr_k50)}
data2["category diversity"]

0.051563691146180916

### k=100

In [9]:
# Instantiate HFT model
ctr_k100 = cornac.models.CTR(
    k=100,
    lambda_u=0.01,
    lambda_v=0.01,
    eta=0.01,
    a=1,
    b=0.01,
    max_iter=10,
    trainable=True,
    verbose=True,
    init_params=None,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[ctr_k100], metrics=metrics, user_based=False
).run()


[CTR] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[CTR] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
CTR | 1.7526 | 4.3511 | 2.0859 | 0.8161 | 0.0632 | 0.0536 | 0.2014 |  0.0777 |  0.1025 |       0.0455 |    0.1892 |   20.0245 |   1.7619



In [10]:
data3 = {"model":"CTR",
        "data":"movielens-100K",
        "k":100,
        "lambda_u":0.01,
        "lambda_v":0.01,
        "eta":0.01,
        "a":1,
        "b":0.01,
        "max_iter":10,
        "MAE":1.7526,"MSE":4.3511,"RMSE":2.0859,"AUC":0.8161,"F1":0.0632,"MAP":0.0536,"MRR":0.2014 ,"NCRR":0.0777,"NDCG":0.1025,"Precision":0.0455,"Recall":0.1892,
        "category diversity":compute_avg_diversity(ctr_k100)}
data3["category diversity"]

0.059648415983780026

### k=10,lambda_v=0.1

In [11]:
# Instantiate HFT model
ctr_v01_k10 = cornac.models.CTR(
    k=10,
    lambda_u=0.01,
    lambda_v=0.1,
    eta=0.01,
    a=1,
    b=0.01,
    max_iter=10,
    trainable=True,
    verbose=True,
    init_params=None,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[ctr_v01_k10], metrics=metrics, user_based=False
).run()


[CTR] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[CTR] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
CTR | 0.8401 | 1.2006 | 1.0957 | 0.9006 | 0.1162 | 0.1025 | 0.2653 |  0.1364 |  0.1978 |       0.0842 |    0.3198 |    6.3560 |   2.1699



In [12]:
data4 = {"model":"CTR",
        "data":"movielens-100K",
        "k":10,
        "lambda_u":0.01,
        "lambda_v":0.1,
        "eta":0.01,
        "a":1,
        "b":0.01,
        "max_iter":10,
        "MAE":0.8401,"MSE":1.2006,"RMSE":1.0957,"AUC":0.9006,"F1":0.1162,"MAP":0.1025,"MRR":0.2653 ,"NCRR":0.1364,"NDCG":0.1978,"Precision":0.0842,"Recall":0.3198,
        "category diversity":compute_avg_diversity(ctr_v01_k10)}
data4["category diversity"]

0.10033744023997579

### k=10, lambda_v=1

In [13]:
# Instantiate HFT model
ctr_v1_k10 = cornac.models.CTR(
    k=10,
    lambda_u=0.01,
    lambda_v=1,
    eta=0.01,
    a=1,
    b=0.01,
    max_iter=10,
    trainable=True,
    verbose=True,
    init_params=None,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[ctr_v1_k10], metrics=metrics, user_based=False
).run()


[CTR] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[CTR] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
CTR | 0.8400 | 1.2001 | 1.0955 | 0.9006 | 0.1164 | 0.1026 | 0.2655 |  0.1365 |  0.1981 |       0.0843 |    0.3207 |    5.3927 |   1.5999



In [14]:
data5 = {"model":"CTR",
        "data":"movielens-100K",
        "k":10,
        "lambda_u":0.01,
        "lambda_v":1,
        "eta":0.01,
        "a":1,
        "b":0.01,
        "max_iter":10,
        "MAE":0.8400,"MSE":1.2001,"RMSE":1.0955,"AUC":0.9006,"F1":0.1164,"MAP":0.1026,"MRR":0.2655 ,"NCRR":0.1365,"NDCG":0.1981,"Precision":0.0843,"Recall":0.3207,
        "category diversity":compute_avg_diversity(ctr_v1_k10)}
data5["category diversity"]

0.10033744023997579

### k=10, lambda_v=10

In [15]:
# Instantiate HFT model
ctr_v10_k10 = cornac.models.CTR(
    k=10,
    lambda_u=0.01,
    lambda_v=10,
    eta=0.01,
    a=1,
    b=0.01,
    max_iter=10,
    trainable=True,
    verbose=True,
    init_params=None,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[ctr_v10_k10], metrics=metrics, user_based=False
).run()


[CTR] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[CTR] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
CTR | 0.8386 | 1.1967 | 1.0939 | 0.9008 | 0.1163 | 0.1018 | 0.2629 |  0.1352 |  0.1974 |       0.0842 |    0.3215 |    5.1215 |   1.7910



In [16]:
data6 = {"model":"CTR",
        "data":"movielens-100K",
        "k":10,
        "lambda_u":0.01,
        "lambda_v":10,
        "eta":0.01,
        "a":1,
        "b":0.01,
        "max_iter":10,
        "MAE":0.8386,"MSE":1.1967,"RMSE":1.0939,"AUC":0.9008,"F1":0.1163,"MAP":0.1018,"MRR":0.2629 ,"NCRR":0.1352,"NDCG":0.1974,"Precision":0.0842,"Recall":0.3215,
        "category diversity":compute_avg_diversity(ctr_v10_k10)}
data6["category diversity"]

0.11239542593885564

### k=10, lambda_v=100

In [17]:
# Instantiate HFT model
ctr_v100_k10 = cornac.models.CTR(
    k=10,
    lambda_u=0.01,
    lambda_v=100,
    eta=0.01,
    a=1,
    b=0.01,
    max_iter=10,
    trainable=True,
    verbose=True,
    init_params=None,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[ctr_v100_k10], metrics=metrics, user_based=False
).run()


[CTR] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[CTR] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
CTR | 0.8336 | 1.1835 | 1.0879 | 0.9014 | 0.1187 | 0.1021 | 0.2581 |  0.1346 |  0.1991 |       0.0859 |    0.3260 |    6.8256 |   2.7199



In [18]:
data7 = {"model":"CTR",
        "data":"movielens-100K",
        "k":10,
        "lambda_u":0.01,
        "lambda_v":100,
        "eta":0.01,
        "a":1,
        "b":0.01,
        "max_iter":10,
        "MAE":0.8336,"MSE":1.1835,"RMSE":1.0879,"AUC":0.9014,"F1":0.1187,"MAP":0.1021,"MRR":0.2581 ,"NCRR":0.1346,"NDCG":0.1991,"Precision":0.0859,"Recall":0.3260,
        "category diversity":compute_avg_diversity(ctr_v100_k10)}
data7["category diversity"]

0.11205451985737414

### k=10, lambda_v=1000

In [19]:
# Instantiate HFT model
ctr_v1000_k10 = cornac.models.CTR(
    k=10,
    lambda_u=0.01,
    lambda_v=1000,
    eta=0.01,
    a=1,
    b=0.01,
    max_iter=10,
    trainable=True,
    verbose=True,
    init_params=None,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[ctr_v1000_k10], metrics=metrics, user_based=False
).run()


[CTR] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[CTR] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
CTR | 0.8269 | 1.1572 | 1.0757 | 0.8959 | 0.1140 | 0.0997 | 0.2545 |  0.1316 |  0.1923 |       0.0828 |    0.3123 |    5.9663 |   1.6421



In [20]:
data8 = {"model":"CTR",
        "data":"movielens-100K",
        "k":10,
        "lambda_u":0.01,
        "lambda_v":1000,
        "eta":0.01,
        "a":1,
        "b":0.01,
        "max_iter":10,
        "MAE":0.8269,"MSE":1.1572,"RMSE":1.0757,"AUC":0.8959,"F1":0.1140,"MAP":0.0997,"MRR":0.2545 ,"NCRR":0.1316,"NDCG":0.1923,"Precision":0.0828,"Recall":0.3123,
        "category diversity":compute_avg_diversity(ctr_v1000_k10)}
data8["category diversity"]

0.12405605576621292

### k=10, lambda_v=10, lambda_u=0.1

In [21]:
# Instantiate HFT model
ctr_u01_v10_k10 = cornac.models.CTR(
    k=10,
    lambda_u=0.1,
    lambda_v=10,
    eta=0.01,
    a=1,
    b=0.01,
    max_iter=10,
    trainable=True,
    verbose=True,
    init_params=None,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[ctr_u01_v10_k10], metrics=metrics, user_based=False
).run()


[CTR] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[CTR] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
CTR | 0.8344 | 1.1791 | 1.0859 | 0.9032 | 0.1187 | 0.1025 | 0.2626 |  0.1357 |  0.2006 |       0.0857 |    0.3310 |    5.5139 |   1.5530



In [22]:
data9 = {"model":"CTR",
        "data":"movielens-100K",
        "k":10,
        "lambda_u":0.1,
        "lambda_v":10,
        "eta":0.01,
        "a":1,
        "b":0.01,
        "max_iter":10,
        "MAE":0.8344,"MSE":1.1791,"RMSE":1.0859,"AUC":0.9032,"F1":0.1187,"MAP":0.1025,"MRR":0.2626 ,"NCRR":0.1357,"NDCG":0.2006,"Precision":0.0857,"Recall":0.3310,
        "category diversity":compute_avg_diversity(ctr_u01_v10_k10)}
data9["category diversity"]

0.0972851299328016

### k=10, lambda_v=10, lambda_u=1

In [23]:
# Instantiate HFT model
ctr_u1_v10_k10 = cornac.models.CTR(
    k=10,
    lambda_u=1,
    lambda_v=10,
    eta=0.01,
    a=1,
    b=0.01,
    max_iter=10,
    trainable=True,
    verbose=True,
    init_params=None,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[ctr_u1_v10_k10], metrics=metrics, user_based=False
).run()


[CTR] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[CTR] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
CTR | 0.8288 | 1.1545 | 1.0745 | 0.9076 | 0.1241 | 0.1090 | 0.2905 |  0.1486 |  0.2123 |       0.0898 |    0.3424 |    5.2899 |   1.3285



In [24]:
data10 = {"model":"CTR",
        "data":"movielens-100K",
        "k":10,
        "lambda_u":1,
        "lambda_v":10,
        "eta":0.01,
        "a":1,
        "b":0.01,
        "max_iter":10,
        "MAE":0.8288,"MSE":1.1545,"RMSE":1.0745,"AUC":0.9076,"F1":0.1241,"MAP":0.1090,"MRR":0.2905 ,"NCRR":0.1486,"NDCG":0.2123,"Precision":0.0898,"Recall":0.3424,
        "category diversity":compute_avg_diversity(ctr_u1_v10_k10)}
data10["category diversity"]

0.11005657125835847

### k=10, lambda_v=10, lambda_u=10

In [25]:
# Instantiate HFT model
ctr_u10_v10_k10 = cornac.models.CTR(
    k=10,
    lambda_u=10,
    lambda_v=10,
    eta=0.01,
    a=1,
    b=0.01,
    max_iter=10,
    trainable=True,
    verbose=True,
    init_params=None,
    seed=123,
)

# Instantiate MSE for evaluation
metrics = [MAE(), RMSE(), MSE(),FMeasure(k=50),Precision(k=50),
           Recall(k=50), NDCG(k=50), NCRR(k=50),
           MRR(),AUC(), MAP()]

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=movieslens100K_ratio_split, models=[ctr_u10_v10_k10], metrics=metrics, user_based=False
).run()


[CTR] Training started!


  0%|          | 0/10 [00:00<?, ?it/s]

Learning completed!

[CTR] Evaluation started!


Rating:   0%|          | 0/18925 [00:00<?, ?it/s]

Ranking:   0%|          | 0/941 [00:00<?, ?it/s]


TEST:
...
    |    MAE |    MSE |   RMSE |    AUC |  F1@50 |    MAP |    MRR | NCRR@50 | NDCG@50 | Precision@50 | Recall@50 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------ + ------ + ------ + ------ + ------- + ------- + ------------ + --------- + --------- + --------
CTR | 0.8451 | 1.1737 | 1.0834 | 0.9124 | 0.1313 | 0.1154 | 0.2990 |  0.1566 |  0.2253 |       0.0949 |    0.3647 |    5.5331 |   1.4931



In [26]:
data11 = {"model":"CTR",
        "data":"movielens-100K",
        "k":10,
        "lambda_u":10,
        "lambda_v":10,
        "eta":0.01,
        "a":1,
        "b":0.01,
        "max_iter":10,
        "MAE":0.8451,"MSE":1.1737,"RMSE":1.0834,"AUC":0.9124,"F1":0.1313,"MAP":0.1154,"MRR":0.2990 ,"NCRR":0.1566,"NDCG":0.2253,"Precision":0.0949,"Recall":0.3647,
        "category diversity":compute_avg_diversity(ctr_u10_v10_k10)}
data11["category diversity"]

0.11489934647504975

In [27]:
data = [data1,data2,data3,data4,data5,data6,data7,data8,data9,data10,data11]
df = pd.DataFrame(columns = data1.keys())
i=0
for d in data:
    df = pd.concat([df, pd.DataFrame(d,columns = list(d.keys()),index=[i])], ignore_index=False)
    i += 1

In [28]:
df

,model,data,k,lambda_u,lambda_v,eta,a,b,max_iter,MAE,...,RMSE,AUC,F1,MAP,MRR,NCRR,NDCG,Precision,Recall,category diversity
0,CTR,movielens-100K,10,0.01,0.01,0.01,1,0.01,10,0.8401,...,1.0957,0.9006,0.1163,0.1025,0.2653,0.1364,0.1979,0.0842,0.3199,0.100337
1,CTR,movielens-100K,50,0.01,0.01,0.01,1,0.01,10,1.2712,...,1.6102,0.8651,0.0976,0.0810,0.2618,0.1217,0.1627,0.0722,0.2631,0.051564
2,CTR,movielens-100K,100,0.01,0.01,0.01,1,0.01,10,1.7526,...,2.0859,0.8161,0.0632,0.0536,0.2014,0.0777,0.1025,0.0455,0.1892,0.059648
3,CTR,movielens-100K,10,0.01,0.10,0.01,1,0.01,10,0.8401,...,1.0957,0.9006,0.1162,0.1025,0.2653,0.1364,0.1978,0.0842,0.3198,0.100337
4,CTR,movielens-100K,10,0.01,1.00,0.01,1,0.01,10,0.8400,...,1.0955,0.9006,0.1164,0.1026,0.2655,0.1365,0.1981,0.0843,0.3207,0.100337
5,CTR,movielens-100K,10,0.01,10.00,0.01,1,0.01,10,0.8386,...,1.0939,0.9008,0.1163,0.1018,0.2629,0.1352,0.1974,0.0842,0.3215,0.112395
6,CTR,movielens-100K,10,0.01,100.00,0.01,1,0.01,10,0.8336,...,1.0879,0.9014,0.1187,0.1021,0.2581,0.1346,0.1991,0.0859,0.3260,0.112055
7,CTR,movielens-100K,10,0.01,1000.00,0.01,1,0.01,10,0.8269,...,1.0757,0.8959,0.1140,0.0997,0.2545,0.1316,0.1923,0.0828,0.3123,0.124056
8,CTR,movielens-100K,10,0.10,10.00,0.01,1,0.01,10,0.8344,...,1.0859,0.9032,0.1187,0.1025,0.2626,0.1357,0.2006,0.0857,0.3310,0.097285
9,CTR,movielens-100K,10,1.00,10.00,0.01,1,0.01,10,0.8288,...,1.0745,0.9076,0.1241,0.1090,0.2905,0.1486,0.2123,0.0898,0.3424,0.110057


In [29]:
df.to_csv('/Users/pigr/Desktop/cornac/cornac/dataset/ctr_movielens_100K.csv',header=-1,index=0)